In [72]:
import pandas as pd
from pulp import *
solver = PULP_CBC_CMD(msg=False)

## 1. Análise e tratamento da planilha
A planilha utilizada foi a [Tabela Brasileira de Composição de Alimentos - 4a edição](https://www.nepa.unicamp.br/taco/tabela.php?ativo=tabela)

Para o escopo do projeto, serão analisados somente os macronutrientes proteína, carboidratos e gorduras (lipídeos); e os micronutrientes vitaminas A e C, minerais cálcio e ferro.

As demais colunas serão removidas.

In [73]:
df = pd.read_excel('taco3.xls')

# Elimina colunas não usadas
df = df.drop(['Unnamed: 2', 'Unnamed: 7'] 
			 	+ df.columns[9:11].tolist() 
				+ df.columns[12:16].tolist() 
				+ df.columns[17:21].tolist() 
				+ df.columns[22:28].tolist(), axis=1)

# Renomeia colunas
df.columns = ['Número do alimento', 'Descrição', 'Energia (kcal)', 
				'Energia (kJ)', 'Proteína (g)', 'Gorduras (g)', 
				'Carboidrato (g)', 'Cálcio (mg)', 'Ferro (mg)', 
				'Vitamina A (mcg)', 'Vitamica C (mg)']

# Elimina linhas redundantes e reseta index
df = df.drop(index=[0, 1], axis=0)
df = df.reset_index(drop=True)

df.head()

,Número do alimento,Descrição,Energia (kcal),Energia (kJ),Proteína (g),Gorduras (g),Carboidrato (g),Cálcio (mg),Ferro (mg),Vitamina A (mcg),Vitamica C (mg)
0,Cereais e derivados,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Arroz, integral, cozido",123.534893,516.86999,2.58825,1.000333,25.80975,5.204,0.262,NaN,NaN
2,2,"Arroz, integral, cru",359.678002,1504.892761,7.323286,1.864833,77.450714,7.818,0.948333,NaN,NaN
3,3,"Arroz, tipo 1, cozido",128.258486,536.633504,2.520817,0.227,28.05985,3.544333,0.076667,NaN,NaN
4,4,"Arroz, tipo 1, cru",357.789273,1496.990319,7.15854,0.335,78.759543,4.414333,0.677747,NaN,NaN


## 2. Estudo das necessidades nutricionais dos indivíduos

Criando um dataframe com as necessidades nutricionais diárias dos indivíduos a partir da tabela passada na descrição do projeto.



In [74]:
lista_pessoas = [
	{'Sexo': 'Homem', 'Peso': 50.0, 'Proteínas': 40.0, 'Carboidratos': 200.0, 'Gorduras': 50.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 60.0, 'Proteínas': 48.0, 'Carboidratos': 240.0, 'Gorduras': 60.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 70.0, 'Proteínas': 56.0, 'Carboidratos': 280.0, 'Gorduras': 70.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 80.0, 'Proteínas': 64.0, 'Carboidratos': 320.0, 'Gorduras': 80.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 90.0, 'Proteínas': 72.0, 'Carboidratos': 360.0, 'Gorduras': 90.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Homem', 'Peso': 100.0, 'Proteínas': 80.0, 'Carboidratos': 400.0, 'Gorduras': 100.0, 'Vitamina A': 900.0, 'Vitamina C': 90.0, 'Cálcio': 1000.0, 'Ferro': 8.0},
	{'Sexo': 'Mulher', 'Peso': 50.0, 'Proteínas': 40.0, 'Carboidratos': 200.0, 'Gorduras': 50.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 55.0, 'Proteínas': 43.0, 'Carboidratos': 215.0, 'Gorduras': 55.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 60.0, 'Proteínas': 46.0, 'Carboidratos': 230.0, 'Gorduras': 60.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 65.0, 'Proteínas': 49.0, 'Carboidratos': 245.0, 'Gorduras': 65.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
	{'Sexo': 'Mulher', 'Peso': 70.0, 'Proteínas': 52.0, 'Carboidratos': 260.0, 'Gorduras': 70.0, 'Vitamina A': 700.0, 'Vitamina C': 75.0, 'Cálcio': 1000.0, 'Ferro': 18.0},
]

df_necessidades = pd.DataFrame(lista_pessoas)
df_necessidades

,Sexo,Peso,Proteínas,Carboidratos,Gorduras,Vitamina A,Vitamina C,Cálcio,Ferro
0,Homem,50.0,40.0,200.0,50.0,900.0,90.0,1000.0,8.0
1,Homem,60.0,48.0,240.0,60.0,900.0,90.0,1000.0,8.0
2,Homem,70.0,56.0,280.0,70.0,900.0,90.0,1000.0,8.0
3,Homem,80.0,64.0,320.0,80.0,900.0,90.0,1000.0,8.0
4,Homem,90.0,72.0,360.0,90.0,900.0,90.0,1000.0,8.0
5,Homem,100.0,80.0,400.0,100.0,900.0,90.0,1000.0,8.0
6,Mulher,50.0,40.0,200.0,50.0,700.0,75.0,1000.0,18.0
7,Mulher,55.0,43.0,215.0,55.0,700.0,75.0,1000.0,18.0
8,Mulher,60.0,46.0,230.0,60.0,700.0,75.0,1000.0,18.0
9,Mulher,65.0,49.0,245.0,65.0,700.0,75.0,1000.0,18.0


## 3. Desenvolvimento do software
Como a abordagem de otimização é livre. Foram escolhidas as seguintes formas de dietas que atinjam as necessidades nutricionais de forma otimizada:
1. Menor quantidade de kcal (perda de peso)

In [75]:
def perda_peso(df, df_necessidades, sexo, peso):
	# Remove todas as linhas que contém strings nas colunas dos nutrientes
	mask = df.loc[:, df.columns != 'Descrição'].applymap(lambda x: isinstance(x, str)).any(axis=1)
	df = df[~mask]
	
	# Cria dicionários dos alimentos e seus respectivos nutrientes
	alimentos = df['Descrição'].tolist()
	kcal = dict(zip(alimentos, df['Energia (kcal)']))
	prt = dict(zip(alimentos, df['Proteína (g)']))
	gord = dict(zip(alimentos, df['Gorduras (g)']))
	carb = dict(zip(alimentos, df['Carboidrato (g)']))
	calc = dict(zip(alimentos, df['Cálcio (mg)']))
	ferro = dict(zip(alimentos, df['Ferro (mg)']))
	vita = dict(zip(alimentos, df['Ferro (mg)']))
	vitc = dict(zip(alimentos, df['Vitamica C (mg)']))
	
	# Encontra a linha de interesse no df e salva-a no df "cliente"
	sexo_col = df_necessidades['Sexo'] == sexo
	peso_col = df_necessidades['Peso'] == peso
	cliente = df_necessidades[sexo_col & peso_col].reset_index(drop=True)

	# Instância do problema
	prob = LpProblem("perda_de_peso", LpMinimize)

	# Cria um dicionário com as variáveis de decisão
	x = LpVariable.dicts('x', alimentos, cat=LpBinary)

	# Função objetivo - a soma das calorias de cada variável (alimento) escolhida
	prob += lpSum([kcal[i] * x[i] for i in alimentos])

	# Restrições
	prob += lpSum([prt[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Proteínas']
	prob += lpSum([gord[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Gorduras']
	prob += lpSum([carb[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Carboidratos']
	prob += lpSum([calc[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Cálcio']
	prob += lpSum([ferro[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Ferro']
	prob += lpSum([vita[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Vitamina A']
	prob += lpSum([vitc[i] * x[i] for i in alimentos]) >= cliente.loc[0, 'Vitamina C']

	prob.solve(solver)
	for alimento in alimentos:
		if x[alimento].value() == 1:
			print(alimento)

perda_peso(df, df_necessidades, 'Homem', 60)

PulpSolverError: Pulp: Error while executing /nfs/homes/viferrei/pucsp/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc